In [1]:
import gzip
from collections import defaultdict
import numpy as np

In [2]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [3]:
cov_type_data = []

for l in readGz("covtype.data.gz"):
    cov_type_data.append(list(l))

In [4]:
np.random.shuffle(cov_type_data)
cov_type_data = cov_type_data[:20000]

In [5]:
for d in cov_type_data:
    if d[54] == 7:
        d[54] = 1
    else:
        d[54] = 0

In [17]:
with open('adult.data') as f:
    adult_data = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
adult_data = [l.split(',') for l in adult_data]

In [18]:
for data in adult_data:
    for i in range(len(data)):
        data[i] = data[i].strip()

In [19]:
np.random.shuffle(adult_data)
adult_data = adult_data[:20000]

In [20]:
def one_hot_enc(k, keys):
    length = len(keys)
    code = [0] * length
    if k != '?' and k in keys:
        idx = keys.index(k)
        code[idx] = 1
    return code

In [21]:
workclass = ['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked'] 
education = ['Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool']
marital_status = ['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse']
occupation = ['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces']
relationship = ['Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried']
race = ['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black']
sex = ['Female', 'Male']
native_country = ['United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands']

In [54]:
adult_onehot = []
for i in range(len(adult_data)):
    d_list = []
    data = adult_data[i]
    d_list.append(int(data[0]))
    d_list += one_hot_enc(data[1], workclass)
    d_list.append(int(data[2]))
    d_list += one_hot_enc(data[3],education)
    d_list.append(int(data[4]))
    d_list += one_hot_enc(data[5],marital_status)
    d_list += one_hot_enc(data[6],occupation)
    d_list += one_hot_enc(data[7],relationship)
    d_list += one_hot_enc(data[8],race)
    d_list += one_hot_enc(data[9],sex)
    d_list.append(int(data[10]))
    d_list.append(int(data[11]))
    d_list.append(int(data[12]))
    d_list += one_hot_enc(data[13],native_country)
    if data[14] == '<=50K':
        d_list.append(0)
    else:
        d_list.append(1)
    adult_onehot.append(d_list)

In [25]:
letter_file = []
with open('letter-recognition.data') as f:
    letter_file = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line

In [26]:
letter_data = []
for data in letter_file:
    d = str(data).rstrip()
    d = d.split(',')
    letter_data.append(d)

In [27]:
for data in letter_data:
    if data[0] in ['A','B','C','D','E','F','G','H','I','J','K','L','M']:
        data[0] = 1
    else:
        data[0] = 0
    for i in range(len(data)):
        data[i] = int(data[i])

In [28]:
np.random.shuffle(letter_data)
letter_data = letter_data[:20000]

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [58]:
cov_XY = np.array(cov_type_data)
adult_XY = np.array(adult_onehot)
letter_XY = np.array(letter_data)

In [33]:
datasets = [cov_XY, adult_XY, letter_XY]
partitions = [0.2, 0.5, 0.8]

In [72]:
for dataset in range(3):
    data = datasets[dataset]
    if dataset == 0:
        print("cov_XY")
    elif dataset == 1:
        print("adult_XY")
    else:
        print("letter_XY")
    for i in range(3):
        if i == 0:
            print("0.2")
        elif i == 1:
            print("0.5")
        else:
            print("0.8")
        part = int(partitions[i]*len(data))
        for trial in range(3):
            np.random.shuffle(data)
            if (dataset != 2):
                X = data[:,:len(data[0])-1]
                Y = data[:,len(data[0])-1]
            else:
                X = data[:,1:]
                Y = data[:,:1].reshape(20000,)
            dt_acc_test = []
            dt_acc_train = []
            dt_acc_val = []
            knn_acc_test = []
            knn_acc_train = []
            knn_acc_val = []
            rf_acc_test = []
            rf_acc_train = []
            rf_acc_val = []
            X_train = X[:part]
            y_train = Y[:part]
            X_test = X[part:len(data)]
            y_test = Y[part:len(data)]
            print("start training dt")
            classifier1 = DecisionTreeClassifier(random_state=0)
            M_list = [5,10,15,20,35]
            clf1 = GridSearchCV(classifier1, {'max_depth':M_list}, return_train_score=True)
            clf1.fit(X_train, y_train)
            dt_acc_val.append(clf1.cv_results_['mean_test_score'][M_list.index(clf1.best_params_['max_depth'])])
            dt_acc_train.append(clf1.cv_results_['mean_train_score'][M_list.index(clf1.best_params_['max_depth'])])
            classifier1.set_params(max_depth=clf1.best_params_['max_depth']).fit(X_train, y_train)
            dt_acc_test.append(classifier1.score(X_test, y_test))
            print("finish training dt")
            
            print("start training knn")
            classifier2 = KNeighborsClassifier(n_neighbors=3)
            N_list = [1,10,50,100,500,1000]
            clf2 = GridSearchCV(classifier2, {'n_neighbors':N_list}, return_train_score=True)
            clf2.fit(X_train, y_train)
            knn_acc_val.append(clf2.cv_results_['mean_test_score'][N_list.index(clf2.best_params_['n_neighbors'])])
            knn_acc_train.append(clf2.cv_results_['mean_train_score'][N_list.index(clf2.best_params_['n_neighbors'])])
            classifier2.set_params(n_neighbors = clf2.best_params_['n_neighbors']).fit(X_train,y_train)
            knn_acc_test.append(classifier2.score(X_test,y_test))
            print("finish training knn")
            
            print("start training rf")
            classifier3 = RandomForestClassifier()
            D_list = [5,15,20,30,35]
            clf3 = GridSearchCV(classifier3, {'max_depth':D_list}, return_train_score=True)
            clf3.fit(X_train,y_train)
            rf_acc_val.append(clf3.cv_results_['mean_test_score'][D_list.index(clf3.best_params_['max_depth'])])
            rf_acc_train.append(clf3.cv_results_['mean_train_score'][D_list.index(clf3.best_params_['max_depth'])])
            classifier3.set_params(max_depth = clf3.best_params_['max_depth']).fit(X_train,y_train)
            rf_acc_test.append(classifier3.score(X_test,y_test))
            print("finish training rf")
                                
        print("val score for dt is: " + str(np.mean(dt_acc_val)))
        print("test score for dt is: " + str(np.mean(dt_acc_test)))
        print("train score for dt is " + str(np.mean(dt_acc_train)))
        print("train score for knn is " + str(np.mean(knn_acc_train)))
        print("test score for knn is " + str(np.mean(knn_acc_test)))
        print("val score for knn is " + str(np.mean(knn_acc_val)))
        print("train score for rf is " + str(np.mean(rf_acc_train)))
        print("test score for rf is " + str(np.mean(rf_acc_test)))
        print("val score for rf is " + str(np.mean(rf_acc_val)))

cov_XY
0.2
start training dt
finish training dt
start training knn
finish training knn
start training rf
finish training rf
start training dt
finish training dt
start training knn
finish training knn
start training rf
finish training rf
start training dt
finish training dt
start training knn
finish training knn
start training rf
finish training rf
val score for dt is: 0.9765
test score for dt is: 0.9756875
train score for dt is 0.9903751715956736
train score for knn is 1.0
test score for knn is 0.9748125
val score for knn is 0.97125
train score for rf is 0.9979998749218613
test score for rf is 0.9790625
val score for rf is 0.97675
0.5
start training dt
finish training dt
start training knn
finish training knn
start training rf
finish training rf
start training dt
finish training dt
start training knn
finish training knn
start training rf
finish training rf
start training dt
finish training dt
start training knn
finish training knn
start training rf
finish training rf
val score for dt i